In [1]:
import json 
import boto3

In [2]:
config_data = json.load(open('config.json'))
aws_access_key_id = config_data['aws_access_key_id']
aws_secret_access_key = config_data['aws_secret_access_key']
config_data.keys()

dict_keys(['aws_access_key_id', 'aws_secret_access_key'])

In [3]:
# create a s3 client
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [4]:
## list all folder in a s3 bucket 
import boto3

def list_s3_folders(bucket_name):
    s3_client = boto3.client('s3')
    
    # List objects in the bucket
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name)
    
    folders = set()
    
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                key = obj['Key']
                # Extract the folder name
                folder = key.split('/')[0]
                folders.add(folder)
    
    return sorted(folders)

bucket_name = 'bindcraft'
folders = list_s3_folders(bucket_name)
print("Folders in S3 bucket:")
for folder in folders:
    print(folder)

Folders in S3 bucket:
2412111959
2412112016
2412120836
2412120937
2412121056
2412121103
2412121113
2412121450
2412142141
2412150205
2412151122
2412151123
2412151601
2412151733
2412241939
2412242139
2412242155
2412250028
2412250101
2412250846
2412251003
2412252245
2412252324
2412252352
2412260103
2412260125
2412260835
2412261730
2412262203
2412270114
2412270144
2412271010
2412271030
2412271144
2412271203
2412271605
2501171452
2501171625
2501180040
2501180802
2501180828
2501180951
2501181203
2501181208
2501192206
2501192248
2501222051
2501230908
2502082313
2502091148
pipeline.png


In [5]:
import boto3
import os

def download_s3_folder(bucket_name, s3_folder, local_dir):
    s3_client = boto3.client('s3')
    
    # Ensure the local directory exists
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    
    # List objects in the specified S3 folder
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder)
    
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                s3_key = obj['Key']
                local_file_path = os.path.join(local_dir, os.path.relpath(s3_key, s3_folder))
                
                # Ensure the local directory exists
                local_file_dir = os.path.dirname(local_file_path)
                if not os.path.exists(local_file_dir):
                    os.makedirs(local_file_dir)
                
                # Download the file
                s3_client.download_file(bucket_name, s3_key, local_file_path)
                print(f"Downloaded {s3_key} to {local_file_path}")

# Example usage
bucket_name = 'bindcraft'
s3_folder = '2502091148'
local_dir = f'./out/bindcraft/{s3_folder}'

download_s3_folder(bucket_name, s3_folder, local_dir)

Downloaded 2502091148/Accepted/7z14_l88_s201849_mpnn4_model2.pdb to ./out/bindcraft/2502091148/Accepted/7z14_l88_s201849_mpnn4_model2.pdb
Downloaded 2502091148/Accepted/7z14_l88_s201849_mpnn7_model2.pdb to ./out/bindcraft/2502091148/Accepted/7z14_l88_s201849_mpnn7_model2.pdb
Downloaded 2502091148/Accepted/Animation/7z14_l88_s201849.html to ./out/bindcraft/2502091148/Accepted/Animation/7z14_l88_s201849.html
Downloaded 2502091148/Accepted/Plots/7z14_l88_s201849_con.png to ./out/bindcraft/2502091148/Accepted/Plots/7z14_l88_s201849_con.png
Downloaded 2502091148/Accepted/Plots/7z14_l88_s201849_i_con.png to ./out/bindcraft/2502091148/Accepted/Plots/7z14_l88_s201849_i_con.png
Downloaded 2502091148/Accepted/Plots/7z14_l88_s201849_i_pae.png to ./out/bindcraft/2502091148/Accepted/Plots/7z14_l88_s201849_i_pae.png
Downloaded 2502091148/Accepted/Plots/7z14_l88_s201849_i_ptm.png to ./out/bindcraft/2502091148/Accepted/Plots/7z14_l88_s201849_i_ptm.png
Downloaded 2502091148/Accepted/Plots/7z14_l88_s201